# Automated ML

Import Dependencies. In the cell below, I imported all the dependencies that needed to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


In [2]:
ws = Workspace.from_config()

experiment_name = 'customer-churn'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
customer-churn,quick-starts-ws-144967,Link to Azure Machine Learning studio,Link to Documentation


## Create or Attach an AmlCompute Cluster

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
This notebook is used to predict whether a customer will change telecommunications provider, something known as "churning".

The dataset is part of the Kaggle Customer Churn Prediction 2020 (https://www.kaggle.com/c/customer-churn-prediction-2020/overview).

The training dataset contains 4250 samples. Each sample contains 19 features and 1 boolean variable "churn" which indicates the class of the sample. The 19 input features and 1 target variable are:

"state", string. 2-letter code of the US state of customer residence
"account_length", numerical. Number of months the customer has been with the current telco provider
"area_code", string="area_code_AAA" where AAA = 3 digit area code.
"international_plan", . The customer has international plan.
"voice_mail_plan", . The customer has voice mail plan.
"number_vmail_messages", numerical. Number of voice-mail messages.
"total_day_minutes", numerical. Total minutes of day calls.
"total_day_calls", numerical. Total minutes of day calls.
"total_day_charge", numerical. Total charge of day calls.
"total_eve_minutes", numerical. Total minutes of evening calls.
"total_eve_calls", numerical. Total number of evening calls.
"total_eve_charge", numerical. Total charge of evening calls.
"total_night_minutes", numerical. Total minutes of night calls.
"total_night_calls", numerical. Total number of night calls.
"total_night_charge", numerical. Total charge of night calls.
"total_intl_minutes", numerical. Total minutes of international calls.
"total_intl_calls", numerical. Total number of international calls.
"total_intl_charge", numerical. Total charge of international calls
"number_customer_service_calls", numerical. Number of calls to customer service
"churn", . Customer churn - target variable.



In [4]:
# Try to load the dataset from the Workspace. 
found = False
key = "customer-churn"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe()
df.describe()

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
count,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000,4250.000000
mean,100.236235,7.631765,180.259600,99.907294,30.644682,200.173906,100.176471,17.015012,200.527882,99.839529,9.023892,10.256071,4.426353,2.769654,1.559059
std,39.698401,13.439882,54.012373,19.850817,9.182096,50.249518,19.908591,4.271212,50.353548,20.093220,2.265922,2.760102,2.463069,0.745204,1.311434
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,73.000000,0.000000,143.325000,87.000000,24.365000,165.925000,87.000000,14.102500,167.225000,86.000000,7.522500,8.500000,3.000000,2.300000,1.000000
50%,100.000000,0.000000,180.450000,100.000000,30.680000,200.700000,100.000000,17.060000,200.450000,100.000000,9.020000,10.300000,4.000000,2.780000,1.000000
75%,127.000000,16.000000,216.200000,113.000000,36.750000,233.775000,114.000000,19.867500,234.700000,113.000000,10.560000,12.000000,6.000000,3.240000,2.000000
max,243.000000,52.000000,351.500000,165.000000,59.760000,359.300000,170.000000,30.540000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000


In [5]:
df.head(5)

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,OH,107,area_code_415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,no
1,NJ,137,area_code_415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,no
2,OH,84,area_code_408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,no
3,OK,75,area_code_415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,no
4,MA,121,area_code_510,no,yes,24,218.2,88,37.09,348.5,108,29.62,212.6,118,9.57,7.5,7,2.03,3,no


## AutoML Configuration

The AutoML experiment will timeout after 15 minutes and uses a maximum of 5 concurrent iterations. It is, however, very possible to adjust those parameters (though a deep learning experiment has a limit of 24 hours).
The primary metric is AUC_weighted (area under the curve weighted), which is the metric I want to optimize. The best-fit model will be chosen based on this metric. The binary classification predicts, whether a customer is going to leave the company or not (given in the lab column "churn"). 
For a detailed explanation, see the Readme file.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="churn",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
customer-churn,AutoML_0442a08e-f9e7-457b-9333-ea80d4ef6c40,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details


In the cell below, I use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
customer-churn,AutoML_0442a08e-f9e7-457b-9333-ea80d4ef6c40,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please c

{'runId': 'AutoML_0442a08e-f9e7-457b-9333-ea80d4ef6c40',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-16T14:27:52.766308Z',
 'endTimeUtc': '2021-05-16T14:46:39.200296Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"customer-churn","subscription_id":"9e65f93e-bdd8-437b-b1e8-0647cd6098f7","resource_group":"aml-quickstarts-144967","workspace_name":"quick-starts-ws-144967","region":"southcentralus","compute_target":"capstone-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"iterations":1000,"primary_metric":"AUC_weighted","task_ty

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, best_model = remote_run.get_output()
print(best_run)
print(best_model)
best_run.get_tags()
best_run_metrics = best_run.get_metrics()

Run(Experiment: customer-churn,
Id: AutoML_0442a08e-f9e7-457b-9333-ea80d4ef6c40_40,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.01,
                                                                                                  min_weight_fraction_leaf=0.0,
              

In [11]:
best_run_metrics

{'precision_score_micro': 0.946587011098751,
 'balanced_accuracy': 0.8181500211888206,
 'average_precision_score_weighted': 0.9613496087172516,
 'precision_score_macro': 0.9583380565502949,
 'log_loss': 0.2244902754625339,
 'weighted_accuracy': 0.9875809321868324,
 'average_precision_score_micro': 0.9669833081290871,
 'AUC_micro': 0.9744475762425219,
 'recall_score_macro': 0.8181500211888206,
 'precision_score_weighted': 0.9479367308030868,
 'norm_macro_recall': 0.6363000423776413,
 'AUC_macro': 0.9254201474141523,
 'accuracy': 0.946587011098751,
 'recall_score_weighted': 0.946587011098751,
 'matthews_correlation': 0.7636903659411005,
 'recall_score_micro': 0.946587011098751,
 'f1_score_macro': 0.8706008090723403,
 'average_precision_score_macro': 0.9249993513852454,
 'f1_score_weighted': 0.9417789209277827,
 'f1_score_micro': 0.946587011098751,
 'AUC_weighted': 0.9254201474141523,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_0442a08e-f9e7-457b-9333-ea80d4ef6c40_40/ac

## Model Deployment

Since this model resulted in a better primary metric than the custom-coded model with optimized hypterparameters, I deployed this model.

In the cell below, I register the model, create an inference config and deploy the model as a web service.

In [12]:
from azureml.core import Model

# Register model
model = best_run.register_model(model_path='outputs/model.pkl', model_name='model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted']})

print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-144967', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-144967'), name=model, id=model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'AUC': '0.9254201474141523'})


In [14]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

env = best_run.get_environment()
env.save_to_directory('./env-dependencies.yml')

inference_config = InferenceConfig(entry_script="scoring.py", environment=env )

In [17]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

In [18]:
service=Model.deploy(workspace=ws,
                    name="deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-14 17:39:49+00:00 Creating Container Registry if not exists.
2021-05-14 17:39:50+00:00 Registering the environment.
2021-05-14 17:39:50+00:00 Use the existing image.
2021-05-14 17:39:50+00:00 Generating deployment configuration.
2021-05-14 17:39:52+00:00 Submitting deployment to compute.
2021-05-14 17:39:54+00:00 Checking the status of deployment deploy-service..
2021-05-14 17:43:06+00:00 Checking the status of inference endpoint deploy-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://974ba161-2a5c-430e-95fb-c052de047cd6.southcentralus.azurecontainer.io/score

swagger URI: 
http://974ba161-2a5c-430e-95fb-c052de047cd6.southcentralus.azurecontainer.io/swagger.json

http://974ba161-2a5c-430e-95fb-c052de047cd6.southcentralus.azurecontainer.io/score
http://974ba161-2a5c-430e-95fb-c052de047cd6.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
df.columns

Index(['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls', 'churn'],
      dtype='object')

In [21]:

import requests
import json

data={"data":
  [{
     'state': "OH", 'account_length': 100, 'area_code': "area_code_415", 'international_plan':"no",
       'voice_mail_plan': "no", 'number_vmail_messages': 26 , 'total_day_minutes': 123.0,
       'total_day_calls': 42, 'total_day_charge': 42.0, 'total_eve_minutes': 42.0,
       'total_eve_calls': 42, 'total_eve_charge': 42.0, 'total_night_minutes': 42.0,
       'total_night_calls': 42, 'total_night_charge': 42.0, 'total_intl_minutes': 42.0,
       'total_intl_calls': 42, 'total_intl_charge': 42.0,
       'number_customer_service_calls': 42
     }, {
     'state': "OH", 'account_length': 100, 'area_code': "area_code_415", 'international_plan':"no",
       'voice_mail_plan': "no", 'number_vmail_messages': 26 , 'total_day_minutes': 123.0,
       'total_day_calls': 42, 'total_day_charge': 42.0, 'total_eve_minutes': 42.0,
       'total_eve_calls': 42, 'total_eve_charge': 42.0, 'total_night_minutes': 42.0,
       'total_night_calls': 42, 'total_night_charge': 42.0, 'total_intl_minutes': 42.0,
       'total_intl_calls': 42, 'total_intl_charge': 42.0,
       'number_customer_service_calls': 42
     }
     ]
  }

input_data=json.dumps(data)

headers={'Content-Type':'application/json'}

result=requests.post(scoring_uri, input_data, headers=headers)
print("Status Code: ", result.status_code)
print("Predicted value: ",result.text)

Status Code:  200
Predicted value:  ["no", "no"]


In the cell below, I print the logs of the web service and delete the service

In [22]:
service.delete()